In [35]:
clear()

In [36]:
import chess.pgn

In [37]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
#     game = chess.pgn.read_game(pgn)
#     print(game)
#     print(game.headers['Event'])
#     board = game.board()
#     for game in (chess.pgn.read_game(pgn)):
    game = chess.pgn.read_game(pgn)
    while game is not None:
#         game = chess.pgn.read_game(pgn)
        board = game.board()
#         print(chess.pgn.read_game(pgn))
        for move in game.mainline_moves():
            board.push(move)
#             print(str(board) +'\n\n')
        game = chess.pgn.read_game(pgn)

KeyboardInterrupt: 

In [48]:
with open('lichess_db_standard_rated_2018-04.pgn') as pgn:
    game = chess.pgn.read_game(pgn)
#     print(game)
    board = game.board()
    for move in game.mainline_moves():
        board.push(move)
        print(str(board) + '\n\n')

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R


r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . . . . .
. . . P P . . .
P P P . . P P P
R N B Q K B N R


r . b q k b n r
p p . p p p p p
. . n . . . . .
. . p . . . . .
. . . . . . . .
. . . P P . . .
P P P . . P P P
R N B Q K B N R


r . b q k b n r
p p . p p p p p
. . n . . . . .
. . p . . . . .
. . . . . . . .
. . . P P N . .
P P P . . P P P
R N B Q K B . R


r . b . k b n r
p p q p p p p p
. . n . . . . .
. . p . . . . .
. . . . . . . .
. . . P P N . .
P P P . . P P P
R N B Q K B . R


r . b . k b n r
p p q p p p p p
. . n . . . . .
. . p . . . . .
. . . . . . . .
. . . P P N . .
P P P . B P P P
R N B Q K . . R


r . b . k b n r
. p q p p p p p
p . n . . . . .
. . p . . . . .
. . . . . . . .
. . . P P 